# Installation

In [ ]:
# @title Mount Google Drive
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# @title Clone
!git clone https://github.com/IAHispano/Applio --branch 3.2.9 --single-branch
%cd /content/Applio

In [ ]:
# @title Install

!sudo apt update
!sudo apt install python3.10
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3
import sys
sys.path.append('/usr/local/lib/python3.10/dist-packages')

rot_47 = lambda encoded_text: "".join(
    [
        (
            chr(
                (ord(c) - (ord("a") if c.islower() else ord("A")) - 47) % 26
                + (ord("a") if c.islower() else ord("A"))
            )
            if c.isalpha()
            else c
        )
        for c in encoded_text
    ]
)

!pip install uv
from IPython.display import clear_output
clear_output()
print("Installing requirements...")
!uv pip install torch==2.7.0 torchvision torchaudio==2.7.0 --upgrade --index-url https://download.pytorch.org/whl/cu128 -q
!uv pip install -r requirements.txt -q
clear_output()
print("Finished installing requirements! ")

In [ ]:
# @title Download models
!python core.py "prerequisites" --models "True" --exe "True"  --pretraineds_hifigan "True"

# Infer


In [ ]:
# @title Download model
# @markdown Hugging Face or Google Drive
model_link = ""  # @param {type:"string"}

!python core.py download --model_link "{model_link}"

In [ ]:
# @title Run Inference
import os

current_dir = os.getcwd()
#@markdown #### نام مدل
model_name = ""  # @param {type:"string"}
model_folder = os.path.join(current_dir, f"logs/{model_name}")

if not os.path.exists(model_folder):
    raise FileNotFoundError(f"Model directory not found: {model_folder}")

files_in_folder = os.listdir(model_folder)
pth_path = next((f for f in files_in_folder if f.endswith(".pth")), None)
index_file = next((f for f in files_in_folder if f.endswith(".index")), None)

if pth_path is None or index_file is None:
    raise FileNotFoundError("No model found.")

pth_file = os.path.join(model_folder, pth_path)
index_file = os.path.join(model_folder, index_file)
#@markdown #### مسیر فایل صوتی
input_path = "/content/example.wav"  # @param {type:"string"}
output_path = "/content/output.wav"
export_format = "WAV"  # @param ['WAV', 'MP3', 'FLAC', 'OGG', 'M4A'] {allow-input: false}
f0_method = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe", "fcpe", "hybrid[rmvpe+fcpe]"] {allow-input: false}
#@markdown #### تنظیم پیچ صدا
f0_up_key = 0  # @param {type:"slider", min:-24, max:24, step:0}
rms_mix_rate = 0.8  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5  # @param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}
clean_strength = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
split_audio = False  # @param{type:"boolean"}
clean_audio = False  # @param{type:"boolean"}
f0_autotune = False  # @param{type:"boolean"}
formant_shift = False # @param{type:"boolean"}
formant_qfrency = 1.0 # @param {type:"slider", min:1.0, max:16.0, step:0.1}
formant_timbre = 1.0 # @param {type:"slider", min:1.0, max:16.0, step:0.1}
embedder_model = "contentvec" # @param ["contentvec", "chinese-hubert-base", "japanese-hubert-base", "korean-hubert-base", "custom"] {allow-input: false}
embedder_model_custom = "" # @param {type:"string"}


# Post-processing effects
if "post_process" not in globals():
  post_process = False
if "reverb" not in globals():
  reverb = False
if "pitch_shift" not in globals():
  pitch_shift = False
if "limiter" not in globals():
  limiter = False
if "gain" not in globals():
  gain = False
if "distortion" not in globals():
  distortion = False
if "chorus" not in globals():
  chorus = False
if "bitcrush" not in globals():
  bitcrush = False
if "clipping" not in globals():
  clipping = False
if "compressor" not in globals():
  compressor = False
if "delay" not in globals():
  delay = False

if "reverb_room_size" not in globals():
  reverb_room_size = 0.5
if "reverb_damping" not in globals():
  reverb_damping = 0.5
if "reverb_wet_gain" not in globals():
  reverb_wet_gain = 0.0
if "reverb_dry_gain" not in globals():
  reverb_dry_gain = 0.0
if "reverb_width" not in globals():
  reverb_width = 1.0
if "reverb_freeze_mode" not in globals():
  reverb_freeze_mode = 0.0

if "pitch_shift_semitones" not in globals():
  pitch_shift_semitones = 0.0

if "limiter_threshold" not in globals():
  limiter_threshold = -1.0
if "limiter_release_time" not in globals():
  limiter_release_time = 0.05

if "gain_db" not in globals():
  gain_db = 0.0

if "distortion_gain" not in globals():
  distortion_gain = 0.0

if "chorus_rate" not in globals():
  chorus_rate = 1.5
if "chorus_depth" not in globals():
  chorus_depth = 0.1
if "chorus_center_delay" not in globals():
  chorus_center_delay = 15.0
if "chorus_feedback" not in globals():
  chorus_feedback = 0.25
if "chorus_mix" not in globals():
  chorus_mix = 0.5

if "bitcrush_bit_depth" not in globals():
  bitcrush_bit_depth = 4

if "clipping_threshold" not in globals():
  clipping_threshold = 0.5

if "compressor_threshold" not in globals():
  compressor_threshold = -20.0
if "compressor_ratio" not in globals():
  compressor_ratio = 4.0
if "compressor_attack" not in globals():
  compressor_attack = 0.001
if "compressor_release" not in globals():
  compressor_release = 0.1

if "delay_seconds" not in globals():
  delay_seconds = 0.1
if "delay_feedback" not in globals():
  delay_feedback = 0.5
if "delay_mix" not in globals():
  delay_mix = 0.5

!python core.py infer --pitch "{f0_up_key}" --volume_envelope "{rms_mix_rate}" --index_rate "{index_rate}" --hop_length "{hop_length}" --protect "{protect}" --f0_autotune "{f0_autotune}" --f0_method "{f0_method}" --input_path "{input_path}" --output_path "{output_path}" --pth_path "{pth_file}" --index_path "{index_file}" --split_audio "{split_audio}" --clean_audio "{clean_audio}" --clean_strength "{clean_strength}" --export_format "{export_format}" --embedder_model "{embedder_model}" --embedder_model_custom "{embedder_model_custom}" --formant_shifting "{formant_shift}" --formant_qfrency "{formant_qfrency}" --formant_timbre "{formant_timbre}" --post_process "{post_process}" --reverb "{reverb}" --pitch_shift "{pitch_shift}" --limiter "{limiter}" --gain "{gain}" --distortion "{distortion}" --chorus "{chorus}" --bitcrush "{bitcrush}" --clipping "{clipping}" --compressor "{compressor}" --delay "{delay}" --reverb_room_size "{reverb_room_size}" --reverb_damping "{reverb_damping}" --reverb_wet_gain "{reverb_wet_gain}" --reverb_dry_gain "{reverb_dry_gain}" --reverb_width "{reverb_width}" --reverb_freeze_mode "{reverb_freeze_mode}" --pitch_shift_semitones "{pitch_shift_semitones}" --limiter_threshold "{limiter_threshold}" --limiter_release_time "{limiter_release_time}" --gain_db "{gain_db}" --distortion_gain "{distortion_gain}" --chorus_rate "{chorus_rate}" --chorus_depth "{chorus_depth}" --chorus_center_delay "{chorus_center_delay}" --chorus_feedback "{chorus_feedback}" --chorus_mix "{chorus_mix}" --bitcrush_bit_depth "{bitcrush_bit_depth}" --clipping_threshold "{clipping_threshold}" --compressor_threshold "{compressor_threshold}" --compressor_ratio "{compressor_ratio}" --compressor_attack "{compressor_attack}" --compressor_release "{compressor_release}" --delay_seconds "{delay_seconds}" --delay_feedback "{delay_feedback}" --delay_mix "{delay_mix}"

from IPython.display import Audio, display, clear_output

output_path = output_path.replace(".wav", f".{export_format.lower()}")
# clear_output()
display(Audio(output_path, autoplay=True))

<style>
  .video-card {
    display: flex;
    align-items: center;
    max-width: 600px;
    margin: 20px auto;
    padding: 15px;
    border-radius: 12px;
    box-shadow: 0 4px 15px rgba(0, 0, 0, 0.1);
    transition: transform 0.3s, box-shadow 0.3s;
    background: linear-gradient(to right, #f8f9fa, #ffffff);
    direction: rtl;
  }
  
  .video-card:hover {
    transform: translateY(-5px);
    box-shadow: 0 10px 25px rgba(0, 0, 0, 0.15);
  }
  
  .thumbnail {
    margin-left: 20px;
    position: relative;
  }
  
  .thumbnail img {
    border-radius: 8px;
    border: none;
    box-shadow: 0 3px 10px rgba(0, 0, 0, 0.2);
  }
  
  .play-icon {
    position: absolute;
    top: 50%;
    left: 50%;
    transform: translate(-50%, -50%);
    width: 60px;
    height: 60px;
    background-color: rgba(255, 0, 0, 0.8);
    border-radius: 50%;
    display: flex;
    justify-content: center;
    align-items: center;
  }
  
  .play-icon:after {
    content: "";
    display: block;
    width: 0;
    height: 0;
    border-top: 10px solid transparent;
    border-bottom: 10px solid transparent;
    border-left: 18px solid white;
    margin-left: 5px;
  }
  
  .video-info {
    flex-grow: 1;
  }
  
  .video-title {
    font-family: 'Vazirmatn', 'Tahoma', sans-serif;
    font-size: 22px;
    font-weight: 700;
    color: #333;
    margin: 0 0 10px 0;
  }
</style>

<a href="http://www.youtube.com/watch?v=W-IKVv5KIgA" target="_blank" style="text-decoration: none; color: inherit;">
  <div class="video-card">
    <div class="thumbnail">
      <img src="http://img.youtube.com/vi/W-IKVv5KIgA/0.jpg" alt="ویدیوی آموزشی" width="240" height="180">
      <div class="play-icon"></div>
    </div>
    <div class="video-info">
      <h3 class="video-title">آموزش ساخت مدل</h3>
      <p style="color: #666; margin: 0;">برای مشاهده ویدیوی آموزشی کلیک کنید</p>
    </div>
  </div>
</a>
